In [1]:
from __future__ import print_function
    
import os
# fixes automatic kernel restart for no obvious reason
os.environ['KMP_DUPLICATE_LIB_OK']='True'


In [2]:
import cv2                 # working with, mainly resizing, images
import numpy as np         # dealing with arrays
import os                  # dealing with directories
from random import shuffle # mixing up or currently ordered data that might lead our network astray in training.
# from tqdm import tqdm 
from tqdm import tqdm_notebook as tqdm
import pprint as pp

In [3]:
TRAIN_DIR = 'train'#'X:/Kaggle_Data/dogs_vs_cats/train/train'
TEST_DIR = 'test'#'X:/Kaggle_Data/dogs_vs_cats/test/test'
IMG_SIZE = 64
LR = 1e-3
train_path = "/Volumes/Macintosh-HD/datasets/tiny-imagenet-200/train"
label_path = "/Volumes/Macintosh-HD/datasets/tiny-imagenet-200"

In [4]:
train_data =[]
label = []
counter = 0
for file in tqdm(os.listdir(train_path)):
    if counter == 20:
        break
    for pic in os.listdir(train_path+"/"+file+"/images"):
        img = cv2.imread(train_path+"/"+file+"/images/"+pic)
        train_data.append(np.array(img))
        label.append(file)
    counter += 1
train_data = np.array(train_data)
# np.save('tiny-imagenet-train.npy', train_data)

print('convert images from uint8 to float64')
train_data = train_data.astype(float)

print('max',np.max(train_data))
print('mean',np.mean(train_data))
print('std',np.std(train_data))

train_data = train_data - np.mean(train_data)
train_data = train_data / np.std(train_data)

print('max after normalisation',np.max(train_data))
print('mean after normalisation',np.mean(train_data))
print('std after normalisation',np.std(train_data))


convert images from uint8 to float64
max 255.0
mean 107.63991454264323
std 66.31287125920697
max after normalisation 2.222194314002005
mean after normalisation 1.7879031588563522e-16
std after normalisation 1.0000000000000018


In [5]:
label = np.array(label)

label = np.reshape(label, (len(label), 1))


In [6]:
lab_id = {}
lab = []
seen = []
labels = []
counter = 0
for l in tqdm(range(label.shape[0])):
    if counter == 20:
        break
    if label[l] in seen:
        continue
    else:
        seen.append(label[l])
        l_id = str(label[l])
        new_dict_entry = {l_id : counter}
        counter += 1 
        lab_id.update(new_dict_entry)
# print(lab_id) 
num_classes = counter
print(num_classes)

20


In [7]:
for l in tqdm(range(label.shape[0])):
    temp = np.zeros(num_classes)
    clss = label[l]
    index = lab_id.get(str(clss))
    temp[index] = 1
    labels.append(temp)

In [8]:
labels = np.array(labels)
# print(labels.shape)

In [9]:
training_data = []
for l in tqdm(range(label.shape[0])):
    training_data.append([train_data[l], labels[l]])
    
shuffle(training_data)

np.save('training_data.npy', training_data)


In [10]:

trn_data = np.load('training_data.npy')
print(trn_data.shape)

(10000, 2)


In [11]:
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression
from tflearn.layers.normalization import batch_normalization

convnet = input_data(shape=[None, IMG_SIZE, IMG_SIZE, 3], name='input')
#       net = layers.conv2d(inputs, 64, [11, 11], 4, padding='VALID', scope='conv1')
#       net = layers_lib.max_pool2d(net, [3, 3], 2, scope='pool1')

#       net = layers.conv2d(net, 192, [5, 5], scope='conv2')
#       net = layers_lib.max_pool2d(net, [3, 3], 2, scope='pool2')

#       net = layers.conv2d(net, 384, [3, 3], scope='conv3')
#       net = layers.conv2d(net, 384, [3, 3], scope='conv4')
#       net = layers.conv2d(net, 256, [3, 3], scope='conv5')
#       net = layers_lib.max_pool2d(net, [3, 3], 2, scope='pool5')
convnet = conv_2d(convnet, 64, [11,11],  4, activation='relu')
# convnet = conv_2d(convnet, 64, [5,5], 2, activation='relu')

convnet = max_pool_2d(convnet,[3,3], 2)

convnet = conv_2d(convnet, 192, [5,5], activation='relu')
convnet = max_pool_2d(convnet,[3,3], 2)

convnet = batch_normalization(convnet)

convnet = conv_2d(convnet, 384, [3,3], activation='relu')

convnet = conv_2d(convnet, 384, [3,3], activation='relu')

convnet = conv_2d(convnet, 256, [3,3], activation='relu')
convnet = max_pool_2d(convnet,[3,3], 2)

convnet = batch_normalization(convnet)

convnet = fully_connected(convnet, 1024, activation='relu')
# convnet = dropout(convnet, 0.8)


convnet = fully_connected(convnet, 1024, activation='relu')
convnet = dropout(convnet, 0.8)
# convnet = dropout(convnet, 0.7)
# convnet = dropout(convnet, 0.6)
# convnet = dropout(convnet, 0.5)


convnet = fully_connected(convnet, num_classes, activation='softmax')
convnet = regression(convnet, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets')

model = tflearn.DNN(convnet, tensorboard_dir='log')


Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [12]:
# print(trn_data)
train = trn_data[:-500]
test = trn_data[-500:]

In [13]:
x = np.array([i[0] for i in train])
# print(x.shape)
y = [i[1] for i in train]

test_x = np.array([i[0] for i in test])
test_y = [i[1] for i in test]

In [14]:
epoch = 0

def train(epoch, epochs):
    print('epoch',epoch)

    for round1 in range(epochs):

        MODEL_NAME = 'test_cnn-{}-{}.model'.format(LR, '2conv-basic-Mon-normInput_float64-bn')

        model.fit({'input': x}, {'targets': y}, n_epoch=1, validation_set=({'input': test_x}, {'targets': test_y}), 
        snapshot_step=100, show_metric=True, run_id=MODEL_NAME)

        epoch += 1
        MODEL_NAME_SAVE = 'models/'+'{}-epochs_{}'.format(MODEL_NAME, epoch)    
        model.save(MODEL_NAME_SAVE)
    
    return epoch

epoch = train(epoch, 3)

Training Step: 446  | total loss: 2.06349 | time: 223.352s
| Adam | epoch: 003 | loss: 2.06349 - acc: 0.3670 -- iter: 9472/9500
Training Step: 447  | total loss: 2.07784 | time: 228.257s
| Adam | epoch: 003 | loss: 2.07784 - acc: 0.3631 | val_loss: 2.19988 - val_acc: 0.3400 -- iter: 9500/9500
--
INFO:tensorflow:/Volumes/System/Downloads/KannaDandamudi/CNN-TinyImagenet/models/test_cnn-0.001-2conv-basic-Mon-normInput_float64-bn.model-epochs_3 is not in all_model_checkpoint_paths. Manually adding it.


In [15]:
epoch = train(epoch, 7)

Training Step: 1489  | total loss: 1.19794 | time: 202.408s
| Adam | epoch: 010 | loss: 1.19794 - acc: 0.6302 -- iter: 9472/9500
Training Step: 1490  | total loss: 1.16058 | time: 207.183s
| Adam | epoch: 010 | loss: 1.16058 - acc: 0.6437 | val_loss: 2.23864 - val_acc: 0.4300 -- iter: 9500/9500
--
INFO:tensorflow:/Volumes/System/Downloads/KannaDandamudi/CNN-TinyImagenet/models/test_cnn-0.001-2conv-basic-Mon-normInput_float64-bn.model-epochs_10 is not in all_model_checkpoint_paths. Manually adding it.


In [16]:
epoch = train(epoch, 10)

Training Step: 2979  | total loss: 0.37705 | time: 214.125s
| Adam | epoch: 020 | loss: 0.37705 - acc: 0.8766 -- iter: 9472/9500
Training Step: 2980  | total loss: 0.41719 | time: 219.082s
| Adam | epoch: 020 | loss: 0.41719 - acc: 0.8671 | val_loss: 2.79123 - val_acc: 0.4480 -- iter: 9500/9500
--
INFO:tensorflow:/Volumes/System/Downloads/KannaDandamudi/CNN-TinyImagenet/models/test_cnn-0.001-2conv-basic-Mon-normInput_float64-bn.model-epochs_20 is not in all_model_checkpoint_paths. Manually adding it.


In [17]:
epoch = train(epoch, 10)

Training Step: 4469  | total loss: 0.20329 | time: 219.975s
| Adam | epoch: 030 | loss: 0.20329 - acc: 0.9360 -- iter: 9472/9500
Training Step: 4470  | total loss: 0.19360 | time: 225.775s
| Adam | epoch: 030 | loss: 0.19360 - acc: 0.9392 | val_loss: 3.39294 - val_acc: 0.4660 -- iter: 9500/9500
--
INFO:tensorflow:/Volumes/System/Downloads/KannaDandamudi/CNN-TinyImagenet/models/test_cnn-0.001-2conv-basic-Mon-normInput_float64-bn.model-epochs_30 is not in all_model_checkpoint_paths. Manually adding it.
